In [1]:
import os
import json
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu
from src.me_to_neurazi import me_to_neurazi

with open(os.path.join('src','kredity.json'), 'r', encoding='utf-8') as kredity:
    kredity = json.loads(kredity.read())

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [2]:
uplne_vsechno = pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet"))

In [3]:
len(uplne_vsechno)

1250624

In [4]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.filter(~pl.col('rok').is_null()).sort(by='rok')
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null())
print(len(df))

1001279
991612


In [5]:
len(df.unique(subset=['100_a','245_a']))

699002

In [129]:
rocni_pocty = df.filter(pl.col('rok').is_between(1800, 2023)).group_by('rok').len().sort('rok').rename({'len':'knih'})
rocni_pocty

rok,knih
i64,u32
1800,1
1801,22
1802,19
1803,22
1804,34
1805,23
1806,16
1807,30
1808,28


In [131]:
rocni_strany = df.filter(pl.col('rok').is_between(1800, 2023)).sort(by="rok").unique(subset=['245_a','100_a']).group_by('rok').agg(pl.col("stran").sum()).sort('rok')
rocni_strany

rok,stran
i64,i64
1800,5
1801,1789
1802,3040
1803,3613
1804,6871
1805,3166
1806,4194
1807,5828
1808,4354


In [8]:
rocni_cetba = 8 * 60 * 365
rocni_cetba

175200

In [9]:
rocni_strany.filter(pl.col('stran') <= rocni_cetba)

rok,stran
datetime[μs],i64
1800-01-01 00:00:00,5
1801-01-01 00:00:00,1789
1802-01-01 00:00:00,3040
1803-01-01 00:00:00,3613
1804-01-01 00:00:00,6871
1805-01-01 00:00:00,3166
1806-01-01 00:00:00,4194
1807-01-01 00:00:00,5828
1808-01-01 00:00:00,4354


In [10]:
rocni_strany.sort(by="stran",descending=True).head(5)

rok,stran
datetime[μs],i64
2008-01-01 00:00:00,2121418
2011-01-01 00:00:00,2120337
2010-01-01 00:00:00,2092612
2018-01-01 00:00:00,2091457
2009-01-01 00:00:00,2081351


In [11]:
rocni_strany.select(pl.col("stran")).tail(1) / rocni_cetba

stran
f64
10.029315


In [133]:
novinky = df.filter(pl.col('rok').is_between(1800, 2023)).filter(pl.col('stran') >= 50).sort('rok').unique(subset=["245_a","100_a"])
novinky = novinky.group_by('rok').len()
novinky = novinky.sort('rok').rename({'len':'novinky'})

In [134]:
novinky.sort(by="novinky",descending=True).head(5)

rok,novinky
i64,u32
2008,10590
2007,10362
2011,10284
2010,10264
2006,10154


In [14]:
celkem_vyslo = df.filter(pl.col('rok').is_between(1850, 2023)).filter(pl.col('stran') >= 50).group_by('rok').agg(pl.col('008').unique().len()).rename({'008':'celkem_vyslo'}).sort('rok')

In [15]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","041.parquet")), left_on="001", right_on="001", how="left")

In [16]:
nepuvodni = df.filter(pl.col('rok').is_between(1850, 2023)).filter(pl.col('stran') >= 50).explode('041_h').filter(pl.col('041_h') != 'cze').group_by('rok').agg(pl.col('008').unique().len()).rename({'008':'preklady'}).sort('rok')
nepuvodni

rok,preklady
i64,u32
1850,22
1851,42
1852,55
1853,50
1854,39
1855,26
1856,26
1857,25
1858,43


In [139]:
preklady = celkem_vyslo.join(nepuvodni, on='rok', how='left')
preklady = preklady.with_columns((pl.col('preklady') / pl.col('celkem_vyslo')).alias('podíl překladů'))
preklady = preklady.select(pl.col(['rok','podíl překladů']))
preklady

rok,podíl překladů
i64,f64
1850,0.196429
1851,0.315789
1852,0.348101
1853,0.352113
1854,0.327731
1855,0.22807
1856,0.240741
1857,0.208333
1858,0.248555


In [145]:
do_grafu = rocni_pocty.join(rocni_strany, on="rok", how="left")
do_grafu = do_grafu.join(novinky, on="rok", how="left")
do_grafu = do_grafu.with_columns((pl.col("stran") / (60 * 8 * 365.25)).alias("let čtení"))
# do_grafu = do_grafu.join(preklady, on='rok', how='left')
do_grafu = do_grafu.melt(id_vars="rok", value_vars=['knih','stran','let čtení','novinky'])
do_grafu = do_grafu.filter(pl.col('variable').is_in(['knih','novinky','podíl překladů']))
do_grafu = do_grafu.with_columns(pl.col('variable').replace_strict({'knih': 'všechny publikace', 'novinky': 'novinky nad 50 stran'}))
do_grafu

rok,variable,value
i64,str,f64
1800,"""všechny publikace""",1.0
1801,"""všechny publikace""",22.0
1802,"""všechny publikace""",19.0
1803,"""všechny publikace""",22.0
1804,"""všechny publikace""",34.0
1805,"""všechny publikace""",23.0
1806,"""všechny publikace""",16.0
1807,"""všechny publikace""",30.0
1808,"""všechny publikace""",28.0


In [19]:
titulek = "Vývoj knižní produkce na českém území"
podtitulek = ["Celkové počty záznamů v České národní bibliografii.","Vedle klasických knih může jít i o katalogy výstav apod."] #,"Angličtina a francouzština (téměř) zmizely jen během 2. světové války,","kdy dominovaly překlady z němčiny. Po osvobození raketově narostl podíl ruštiny."]
kredit = "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025"

In [143]:
do_grafu

rok,variable,value
datetime[μs],str,f64
1800-01-01 00:00:00,"""všechny publikace""",1.0
1801-01-01 00:00:00,"""všechny publikace""",22.0
1802-01-01 00:00:00,"""všechny publikace""",19.0
1803-01-01 00:00:00,"""všechny publikace""",22.0
1804-01-01 00:00:00,"""všechny publikace""",34.0
1805-01-01 00:00:00,"""všechny publikace""",23.0
1806-01-01 00:00:00,"""všechny publikace""",16.0
1807-01-01 00:00:00,"""všechny publikace""",30.0
1808-01-01 00:00:00,"""všechny publikace""",28.0


In [125]:
alt.Chart(do_grafu.head(130).to_pandas()).mark_line().encode(alt.X('rok:T'),alt.Y('value:Q'))

alt.Chart(...)

In [147]:
vyvoj_produkce = alt.Chart(
    alt_friendly(do_grafu),
    title=alt.Title('Vývoj knižní produkce na českém území'),
    width=kredity['sirka'],
    height=kredity['vyska_nizkych'] * 1,
    autosize=alt.AutoSizeParams(
        type='pad',
        contains='padding')
).mark_bar(width=1, color='#DB842F').encode(
    alt.X('rok:T', title=None, axis=alt.Axis(tickCount=5)),
    alt.Y(
        'value:Q', 
        title=None, 
        axis=alt.Axis(orient='right', domainOpacity=0, tickColor='#DCDDD6', format='s', labelExpr='split(datum.label, "k")[0] + " tis."'), 
        scale=alt.Scale(domainMin=1)),
    alt.Row('variable:N', title=None, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='start', labelOrient='top', labelFont='Asap'), sort=['všechny publikace','novinky nad 50 stran','podíl překladů'])
).resolve_scale(
    y='independent'
).configure_view(
    stroke='transparent'
).configure_axisX(
    grid=False, domain=False
).configure_axisY(
    grid=False, domain=False
).resolve_axis(
    x="independent",
    y="independent"
).resolve_scale(
    y="shared"
)

vyvoj_produkce

alt.Chart(...)

In [24]:
vyvoj_produkce['title']['text']

'Vývoj knižní produkce na českém území'

In [97]:
me_to_neurazi(vyvoj_produkce, kredity=kredity['default'], soubor="01_vyvoj_produkce")

<figure>
    <a href="https://data.irozhlas.cz/knihy-grafy/01_vyvoj_produkce.svg" target="_blank">
    <img src="https://data.irozhlas.cz/knihy-grafy/01_vyvoj_produkce.svg" width="100%" alt="Graf s titulkem „Vývoj knižní produkce na českém území“. Další texty by měly být čitelné ze zdrojového souboru SVG." />
    </a>
    </figure>


In [167]:
alt.Chart(alt_friendly(
    do_grafu.filter(pl.col('variable') == 'všechny publikace').with_columns(
        pl.when(pl.col('rok').is_between(1801,1850)).then(pl.lit("první polovina 19. století")).when(pl.col('rok').is_between(1851,1900)).then(pl.lit("druhá polovina 19. století")).when(pl.col('rok').is_between(1901,1950)).then(pl.lit("první polovina 20. století")).when(pl.col('rok').is_between(1951,2000)).then(pl.lit("druhá polovina 20. století")).when(pl.col('rok').is_between(2001,2023)).then(pl.lit("10. století")).alias("kdy")
    )
)).mark_bar().encode(
    alt.X("rok:T"),
    alt.Y("value"),
    alt.Row("kdy:N")
).resolve_scale(y="independent",x="independent").resolve_axis(x="independent")

alt.Chart(...)

In [149]:
do_grafu

rok,variable,value
i64,str,f64
1800,"""všechny publikace""",1.0
1801,"""všechny publikace""",22.0
1802,"""všechny publikace""",19.0
1803,"""všechny publikace""",22.0
1804,"""všechny publikace""",34.0
1805,"""všechny publikace""",23.0
1806,"""všechny publikace""",16.0
1807,"""všechny publikace""",30.0
1808,"""všechny publikace""",28.0
